<a href="https://colab.research.google.com/github/Refactory-Capstone-Project/Loan-default-prediction-challenge-/blob/main/LoanPredictionSubmission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas-profiling xgboost gradio

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from ydata_profiling import ProfileReport

In [3]:
!wget https://github.com/Refactory-Capstone-Project/Loan-default-prediction-challenge-/raw/main/Loan_Prediction.zip

--2024-04-23 13:24:02--  https://github.com/Refactory-Capstone-Project/Loan-default-prediction-challenge-/raw/main/Loan_Prediction.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Refactory-Capstone-Project/Loan-default-prediction-challenge-/main/Loan_Prediction.zip [following]
--2024-04-23 13:24:02--  https://raw.githubusercontent.com/Refactory-Capstone-Project/Loan-default-prediction-challenge-/main/Loan_Prediction.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1627991 (1.6M) [application/zip]
Saving to: ‘Loan_Prediction.zip.3’

Loan_Prediction.zip 100%[===================>]   1.55M  -

In [4]:
!unzip /content/Loan_Prediction.zip.2

Archive:  /content/Loan_Prediction.zip.2
replace SampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: SampleSubmission.csv    
replace testdemographics.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: testdemographics.csv    
replace testperf.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: testperf.csv            
replace testprevloans.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: testprevloans.csv       
replace traindemographics.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: traindemographics.csv   
replace trainperf.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: trainperf.csv           
replace trainprevloans.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: trainprevloans.csv      


# Demographic data (traindemographics.csv)

In [5]:
train_demo = pd.read_csv("/content/traindemographics.csv.2")
train_demo.head()

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients
0,8a858e135cb22031015cbafc76964ebd,1973-10-10 00:00:00.000000,Savings,3.319219,6.528604,GT Bank,NaN,NaN,NaN
1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21 00:00:00.000000,Savings,3.325598,7.119403,Sterling Bank,NaN,Permanent,NaN
2,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,NaN,NaN
3,8a858efd5ca70688015cabd1f1e94b55,1991-07-19 00:00:00.000000,Savings,3.362850,6.642485,GT Bank,NaN,Permanent,NaN
4,8a858e785acd3412015acd48f4920d04,1982-11-22 00:00:00.000000,Savings,8.455332,11.971410,GT Bank,NaN,Permanent,NaN


In [6]:
train_demo.dtypes

customerid                     object
birthdate                      object
bank_account_type              object
longitude_gps                 float64
latitude_gps                  float64
bank_name_clients              object
bank_branch_clients            object
employment_status_clients      object
level_of_education_clients     object
dtype: object

In [7]:
train_demo['birthdate'] = pd.to_datetime(train_demo['birthdate'])

train_demo["birthdate_year"] = pd.to_datetime(train_demo["birthdate"]).dt.year
train_demo["birthdate_month"] = pd.to_datetime(train_demo["birthdate"]).dt.month
train_demo["birthdate_day"] = pd.to_datetime(train_demo["birthdate"]).dt.day
train_demo.dtypes

customerid                            object
birthdate                     datetime64[ns]
bank_account_type                     object
longitude_gps                        float64
latitude_gps                         float64
bank_name_clients                     object
bank_branch_clients                   object
employment_status_clients             object
level_of_education_clients            object
birthdate_year                         int32
birthdate_month                        int32
birthdate_day                          int32
dtype: object

In [8]:
train_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4346 entries, 0 to 4345
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   customerid                  4346 non-null   object        
 1   birthdate                   4346 non-null   datetime64[ns]
 2   bank_account_type           4346 non-null   object        
 3   longitude_gps               4346 non-null   float64       
 4   latitude_gps                4346 non-null   float64       
 5   bank_name_clients           4346 non-null   object        
 6   bank_branch_clients         51 non-null     object        
 7   employment_status_clients   3698 non-null   object        
 8   level_of_education_clients  587 non-null    object        
 9   birthdate_year              4346 non-null   int32         
 10  birthdate_month             4346 non-null   int32         
 11  birthdate_day               4346 non-null   int32       

In [12]:
train_demo[train_demo['customerid'].duplicated(keep=False)].sort_values('customerid')

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients,birthdate_year,birthdate_month,birthdate_day
1414,8a858e625c8d993a015c938f829f77ee,1988-12-20,Savings,5.768333,5.561992,First Bank,NaN,Permanent,NaN,1988,12,20
1928,8a858e625c8d993a015c938f829f77ee,1988-12-20,Savings,5.768333,5.561992,First Bank,NaN,Permanent,NaN,1988,12,20
445,8a858e6c5c88d145015c8b9627cd5a48,1979-09-30,Savings,3.367008,6.497313,Sterling Bank,NaN,Permanent,NaN,1979,9,30
1090,8a858e6c5c88d145015c8b9627cd5a48,1979-09-30,Savings,3.367008,6.497313,Sterling Bank,NaN,Permanent,NaN,1979,9,30
1996,8a858ec65cc6352b015cc64525ea0763,1985-01-30,Savings,3.845728,7.411737,GT Bank,NaN,Permanent,NaN,1985,1,30
1520,8a858ec65cc6352b015cc64525ea0763,1985-01-30,Savings,3.845728,7.411737,GT Bank,NaN,Permanent,NaN,1985,1,30
272,8a858edd57f790040157ffe9b6ed3fbb,1988-01-18,Other,3.782563,7.171356,First Bank,NaN,Permanent,Secondary,1988,1,18
517,8a858edd57f790040157ffe9b6ed3fbb,1988-01-18,Other,3.782563,7.171356,First Bank,NaN,Permanent,Secondary,1988,1,18
4126,8a858f1e5baffcc9015bb02b505f180d,1983-04-06,Savings,6.969350,4.818535,GT Bank,NaN,Permanent,NaN,1983,4,6
3021,8a858f1e5baffcc9015bb02b505f180d,1983-04-06,Savings,6.969350,4.818535,GT Bank,NaN,Permanent,NaN,1983,4,6


In [13]:
train_demo = train_demo.drop_duplicates(subset='customerid', keep='first', inplace=False)

In [15]:
train_demo['bank_account_type'].value_counts()

bank_account_type
Savings    3414
Other       864
Current      56
Name: count, dtype: int64

# Performance data (trainperf.csv)

This is the repeat loan that the customer has taken for which we need to predict the performance of. Basically, we need to predict if whether this loan would default given all previous loans and demographics of a customer

In [9]:
train_perf = pd.read_csv('/content/trainperf.csv')
train_perf.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,referredby,good_bad_flag
0,8a2a81a74ce8c05d014cfb32a0da1049,301994762,12,2017-07-25 08:22:56.000000,2017-07-25 07:22:47.000000,30000.0,34500.0,30,NaN,Good
1,8a85886e54beabf90154c0a29ae757c0,301965204,2,2017-07-05 17:04:41.000000,2017-07-05 16:04:18.000000,15000.0,17250.0,30,NaN,Good
2,8a8588f35438fe12015444567666018e,301966580,7,2017-07-06 14:52:57.000000,2017-07-06 13:52:51.000000,20000.0,22250.0,15,NaN,Good
3,8a85890754145ace015429211b513e16,301999343,3,2017-07-27 19:00:41.000000,2017-07-27 18:00:35.000000,10000.0,11500.0,15,NaN,Good
4,8a858970548359cc0154883481981866,301962360,9,2017-07-03 23:42:45.000000,2017-07-03 22:42:39.000000,40000.0,44000.0,30,NaN,Good


# Previous loans data (trainprevloans.csv)

This dataset contains all previous loans that the customer had prior to the loan above that we want to predict the performance of. Each loan will have a different systemloanid, but the same customerid for each customer.

In [11]:
train_prevloans = pd.read_csv('/content/trainprevloans.csv')
train_prevloans.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,closeddate,referredby,firstduedate,firstrepaiddate
0,8a2a81a74ce8c05d014cfb32a0da1049,301682320,2,2016-08-15 18:22:40.000000,2016-08-15 17:22:32.000000,10000.0,13000.0,30,2016-09-01 16:06:48.000000,NaN,2016-09-14 00:00:00.000000,2016-09-01 15:51:43.000000
1,8a2a81a74ce8c05d014cfb32a0da1049,301883808,9,2017-04-28 18:39:07.000000,2017-04-28 17:38:53.000000,10000.0,13000.0,30,2017-05-28 14:44:49.000000,NaN,2017-05-30 00:00:00.000000,2017-05-26 00:00:00.000000
2,8a2a81a74ce8c05d014cfb32a0da1049,301831714,8,2017-03-05 10:56:25.000000,2017-03-05 09:56:19.000000,20000.0,23800.0,30,2017-04-26 22:18:56.000000,NaN,2017-04-04 00:00:00.000000,2017-04-26 22:03:47.000000
3,8a8588f35438fe12015444567666018e,301861541,5,2017-04-09 18:25:55.000000,2017-04-09 17:25:42.000000,10000.0,11500.0,15,2017-04-24 01:35:52.000000,NaN,2017-04-24 00:00:00.000000,2017-04-24 00:48:43.000000
4,8a85890754145ace015429211b513e16,301941754,2,2017-06-17 09:29:57.000000,2017-06-17 08:29:50.000000,10000.0,11500.0,15,2017-07-14 21:18:43.000000,NaN,2017-07-03 00:00:00.000000,2017-07-14 21:08:35.000000
